In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import csv
import os

1. preprocess data

In [2]:
train_data = pd.read_csv("dataset/train.csv", encoding='big5')
test_data = pd.read_csv("dataset/test.csv", encoding='big5')
sample_submission = pd.read_csv("dataset/sampleSubmission.csv", encoding='big5')

In [3]:
train_data = train_data.iloc[:, 3:]
train_data.shape

(4320, 24)

In [4]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,14,14,14,13,12,12,12,12,15,17,...,22,22,21,19,17,16,15,15,15,15
1,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


In [5]:
train_data.replace("NR", 0 , inplace=True)

In [6]:
# month : 12, pollution_source : 18, days : 20, hours each day : 24
train_each_month = np.zeros((12, 18, 24 * 20), dtype=np.float32)
for i in range(train_each_month.shape[0]):
    for j in range(20):
        train_each_month[i, :, j * 24 : j * 24 + 24] = train_data.iloc[i + j * 18:i + 18 + j * 18, :]

In [7]:
train_each_month[0, 0, :48]

array([14., 14., 14., 13., 12., 12., 12., 12., 15., 17., 20., 22., 22.,
       22., 22., 22., 21., 19., 17., 16., 15., 15., 15., 15., 16., 15.,
       15., 14., 14., 15., 16., 16., 17., 20., 22., 23., 24., 24., 24.,
       24., 23., 21., 20., 19., 18., 18., 18., 18.], dtype=float32)

In [ ]:
rows = 471 * 12
columns = 18 * 9
index = 0
train_x = np.zeros((rows, columns))
train_y = np.zeros((rows, 1))
for i in range(train_data.shape[0]):
    for j in range(train_data.shape[1]):
        if j >= 15:
            continue
        train_x[index] = train_data.iloc[i, j:(j+9)]
        train_y[index] = train_data.iloc[i, j+9]
        index +=1

[reference link](https://github.com/maplezzz/NTU_ML2017_Hung-yi-Lee_HW/tree/master/HW1)